# Generate enclosed tessellation

This notebook generates enclosed tessellation for the whole Great Britain within [enclosures](Generate_enclosures).

Note: An algorithm to generate enclosed tessellation has been implemented in momepy 0.4.0 within [`momepy.Tessellation`](http://docs.momepy.org/en/latest/generated/momepy.Tessellation.html#momepy.Tessellation).

## Start dask

Enclosed tessellation is a `dask`-based parallelised algorithm, so let’s start a client first.

In [1]:
from dask.distributed import Client

client = Client()
client

Client Scheduler: tcp://127.0.0.1:38895 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 7 Cores: 28 Memory: 84.28 GB


Function for `enclosed_tessellation` is loaded from `tessellation.py` in the same directory. In the future, it will be part of `momepy`.

In [2]:
import time

import numpy as np
import geopandas as gpd
from tqdm.notebook import tqdm

from tessellation import enclosed_tessellation

The function loads [pre-chunked](chunk_uk) parquet files of buildings and enclosures, generates enclosed tessellation as saves it to another parquet file. 

In [ ]:
%%time
for chunk_id in tqdm(range(103), total=103):
    blg = gpd.read_parquet(f'../../urbangrammar_samba/spatial_signatures/buildings/blg_{chunk_id}.pq')
    enclosures = gpd.read_parquet(f'../../urbangrammar_samba/spatial_signatures/enclosures/encl_{chunk_id}.pq')
    tess = enclosed_tessellation(buildings=blg, enclosures=enclosures.geometry.values, unique_id="uID", enclosure_id="enclosureID")
    tess['enclosureID'] = np.take(enclosures.enclosureID, tess.enclosureID).values
    tess.to_parquet(f'../../urbangrammar_samba/spatial_signatures/tessellation/tess_{chunk_id}.pq')
    client.restart()
    time.sleep(5)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a production setting unless you are able to rewrite your Parquet/Feather files.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')
  """
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
d

In [5]:
client.close()